# Data Conversion for SPSS Analysis 

## Notebook Summary
- Loads cleaned experiment tables generated by the preprocessing notebooks in this repository.
- Computes trial-level metrics (correctness, layer-change behavior, and timing features) for SPSS analysis.
- Exports multiple SPSS-ready CSV datasets for complete data and experiment-only data (training blocks excluded).
- Builds additional aggregate and wide-format tables for condition-based and repeated-measures statistics.

## Setup Shared Utilities
Load project-wide helper code and path/configuration variables from `functions.ipynb`.

In [1]:
# Imports and utilities from functions.ipynb

%run functions.ipynb

## Load Input Tables
Read the prepared experiment, result, and number-display CSV files that are used for SPSS export.

In [2]:
data_complete = pd.read_csv(rf'{export_data}data_experiment.csv', sep=";")
results_complete = pd.read_csv(rf'{export_data}results_valid_all.csv', sep=";")

numbers_display = pd.read_csv(rf'{export_data}numberDisplayed_all.csv', sep= ';')

## Build Trial-Level SPSS Exports
Compute layer-change and timing features, derive correctness/error variables, and export trial-level SPSS tables (including error-focused and layer-change outputs).

In [14]:
# display(data_complete)

display('Results:')
display(results_complete)

results_experiment = results_complete[results_complete['BlockId'] >= 0]
# compute LayerChanges

data_complete['LayerChange'] = data_complete['Layer'].replace('-', np.NaN).astype(float).diff()
grouped_p_tNr = data_complete[(data_complete['LayerChange'] != 0) & (data_complete['LayerChange'] != np.NaN)].groupby(['ProbandId', 'TrialNumber']).count()['LayerChange'].reset_index()

merged = pd.merge(results_complete, grouped_p_tNr, on=['ProbandId', 'TrialNumber'], how='left')
merged_experiment = merged[merged['BlockId'] >= 0]

display('LayerStats:')
display(merged)

numbersCount_p_tNr = numbers_display.groupby(['ProbandId', 'TrialNumber']).count()['Duration_ShowNumber'].reset_index()
numbersTime_p_tNr = numbers_display.groupby(['ProbandId', 'TrialNumber']).mean()['Duration_ShowNumber'].reset_index()
centerCount_p_tNr = numbers_display.groupby(['ProbandId', 'TrialNumber']).count()['Duration_FindCenter'].reset_index()
centerTime_p_tNr = numbers_display.groupby(['ProbandId', 'TrialNumber']).mean()['Duration_FindCenter'].reset_index()


merged = pd.merge(merged, numbersCount_p_tNr, on=['ProbandId', 'TrialNumber'], how='left')
merged = pd.merge(merged, numbersTime_p_tNr, on=['ProbandId', 'TrialNumber'], how='left')
merged = pd.merge(merged, centerCount_p_tNr, on=['ProbandId', 'TrialNumber'], how='left')
merged = pd.merge(merged, centerTime_p_tNr, on=['ProbandId', 'TrialNumber'], how='left')

display('Numbers:')
display(merged)

results_spss = pd.DataFrame()
results_spss['ProbandId'] = results_complete['ProbandId']
results_spss['BlockId'] = results_complete['BlockId']
results_spss['TrialNumber'] = results_complete['TrialNumber']
results_spss['TrialId'] = results_complete['TrialId']
results_spss['Condition'] = results_complete['Condition']
results_spss['ConditionId'] = results_spss['Condition'].map(condition_names.index)
results_spss['Visual'] = 0
results_spss['Tactile'] = 0
# visual feedback = 2, combined = 3 
results_spss.loc[results_spss['ConditionId'] > 1, 'Visual'] = 1
# tactile feedback = 1
results_spss.loc[results_spss['ConditionId'] == 1, 'Tactile'] = 1
# combined = 3
results_spss.loc[results_spss['ConditionId'] == 3, 'Tactile'] = 1
results_spss['ResultLayerCorrect'] = results_complete['ResultLayerCorrect'].astype(int)
results_spss['ResultLayerDifference'] = results_complete['ResultLayerDifference']
results_spss['ResultLayerDifference_Abs'] = results_complete['ResultLayerDifference'].abs()
results_spss['ResultLayerDifference_Proz'] = results_spss['ResultLayerDifference_Abs'] / 7.0
results_spss['ResultNumberCorrect'] = results_complete['ResultNumberCorrect'].astype(int)
results_spss['ResultNumberDifference'] = results_complete['ResultNumberDifference']
results_spss['TrialDurationS'] = results_complete['DurationMS_Cleaned']
results_spss['LayerChangeCount'] = merged['LayerChange']
results_spss['ShowNumberCount'] = merged['Duration_ShowNumber_x']
results_spss['ShowNumberDiff'] = merged['LayerChange'] - merged['Duration_ShowNumber_x']
results_spss['ShowNumberDurationMS'] = merged['Duration_ShowNumber_y'] * 1000
results_spss['CenterDurationMS'] = merged['Duration_FindCenter_y'] * 1000

results_spss['SumCorrect'] = results_spss['ResultLayerCorrect'] + results_spss['ResultNumberCorrect']
results_spss['BothCorrect'] = 0
results_spss['NoneCorrect'] = 0
results_spss['OneCorrect'] = 0

results_spss.loc[results_spss['SumCorrect'] == 2, 'BothCorrect'] = 1
results_spss.loc[results_spss['SumCorrect'] == 1, 'OneCorrect'] = 1
results_spss.loc[results_spss['SumCorrect'] == 0, 'NoneCorrect'] = 1


results_spss_experiment = results_spss[results_spss['BlockId'] >= 0]

display('SPSS EXPORT:')
display(results_spss)

results_spss.to_csv(rf'{export_data_spss}results_complete.csv', sep= ";")
results_spss_experiment.to_csv(rf'{export_data_spss}results_experiment.csv', sep= ";")

results_spss_onlyerror = pd.DataFrame()
results_spss_onlyerror['BlockId'] = results_spss['BlockId']
results_spss_onlyerror['Condition'] = results_spss['Condition']
results_spss_onlyerror['ConditionId'] = results_spss['ConditionId']
results_spss_onlyerror['TrialId'] = results_spss['TrialId']
results_spss_onlyerror['ResultErrorOR'] = 0

results_spss_onlyerror.loc[results_spss['SumCorrect'] != 2, 'ResultErrorOR'] = 1

display('SPSS EXPORT (ERROR Only):')
display(results_spss_onlyerror)

results_spss_onlyerror_experiment = results_spss_onlyerror[results_spss_onlyerror['BlockId'] >= 0]

results_spss_onlyerror.to_csv(rf'{export_data_spss}results_onlyError.csv', sep= ";")
results_spss_onlyerror_experiment.to_csv(rf'{export_data_spss}results__onlyError_experiment.csv', sep= ";")

results_spss_errorSum = results_spss_onlyerror_experiment.groupby(['Condition', 'TrialId']).sum()['ResultErrorOR'].reset_index()

results_spss_errorSum['ConditionId'] = results_spss_errorSum['Condition'].map(condition_names.index)

cId = results_spss_errorSum.pop('ConditionId')
results_spss_errorSum.insert(1, cId.name, cId)

display('SPSS EXPORT (ERROR Sum):')
display(results_spss_errorSum)

results_spss_errorSum.to_csv(rf'{export_data_spss}results_errorSumOR.csv', sep= ";")


merged.to_csv(rf'{export_data_spss}layerChanges_complete.csv', sep= ";")
merged_experiment.to_csv(rf'{export_data_spss}layerChanges_experiment.csv', sep= ";")

# compute Layer Changes per layer
grouped_p_tNr_l = data_complete[(data_complete['Layer'] != '-') & (data_complete['LayerChange'] != 0) & (data_complete['LayerChange'] != np.NaN)].groupby(['ProbandId', 'TrialNumber', 'Layer']).count()['LayerChange'].reset_index()
merged_layer = pd.merge(results_complete, grouped_p_tNr_l, on=['ProbandId', 'TrialNumber'], how='left')
merged_layer_experiment = merged_layer[merged_layer['BlockId'] >= 0]


display('LayerStats per Layer:')
display(merged_layer)

merged_layer.to_csv(rf'{export_data_spss}layerChanges_layer_complete.csv', sep= ";")
merged_layer_experiment.to_csv(rf'{export_data_spss}layerChanges_layer_experiment.csv', sep= ";")


'Results:'

,BlockId,CommitResultDate,Condition,EndInteractionDate,ExpectedResultLayer,ExpectedResultNumber,LayerNumberConfiguration,ProbandId,ProvidedResultLayer,ProvidedResultNumber,...,TrialNumber,CleanedUpInteractionStart_DT,CleanedUpInteractionEnd_DT,ResultNumberDifference,ResultLayerDifference,ResultLayerCorrect,ResultNumberCorrect,DurationMS,DurationMS_Cleaned,countCondition
0,-4,2022-08-01T14:01:37.953Z,Combined Feedback,2022-08-01T14:01:30.874Z,3,176,"[131, 144, 176, 153, 147, 135, 114]",2,2,153,...,0,2022-08-01 13:59:15.117,2022-08-01 14:01:30.874,-23,-1,False,False,135.757,135.757,0
1,-4,2022-08-01T14:02:26.995Z,Combined Feedback,2022-08-01T14:02:22.682Z,7,158,"[149, 118, 145, 137, 147, 146, 158]",2,4,147,...,0,2022-08-01 14:01:37.954,2022-08-01 14:02:22.682,-11,-3,False,False,44.728,44.728,1
2,-3,2022-08-01T14:03:44.338Z,Visual Feedback,2022-08-01T14:03:39.613Z,2,161,"[150, 161, 134, 151, 128, 149, 127]",2,2,161,...,0,2022-08-01 14:02:33.306,2022-08-01 14:03:39.613,0,0,True,True,66.307,66.307,0
3,-3,2022-08-01T14:04:34.107Z,Visual Feedback,2022-08-01T14:04:29.378Z,4,150,"[132, 144, 148, 150, 149, 147, 130]",2,4,150,...,0,2022-08-01 14:03:44.339,2022-08-01 14:04:29.378,0,0,True,True,45.039,45.039,1
4,-2,2022-08-01T14:07:33.355Z,Tactile Feedback,2022-08-01T14:07:23.779Z,6,160,"[139, 144, 128, 136, 156, 160, 137]",2,3,160,...,0,2022-08-01 14:05:08.405,2022-08-01 14:07:23.779,0,-3,False,True,135.374,135.374,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927,3,2022-08-23T11:27:13.692Z,No Feedback,2022-08-23T11:27:09.628Z,7,159,"[153, 122, 128, 144, 140, 154, 159]",23,7,159,...,79,2022-08-23 11:26:33.878,2022-08-23 11:27:08.564,0,0,True,True,37.272,34.686,501
1928,3,2022-08-23T11:27:49.516Z,No Feedback,2022-08-23T11:27:45.172Z,5,157,"[146, 142, 133, 156, 157, 128, 138]",23,5,157,...,80,2022-08-23 11:27:15.467,2022-08-23 11:27:44.655,0,0,True,True,31.479,29.188,502
1929,3,2022-08-23T11:28:51.795Z,No Feedback,2022-08-23T11:28:46.148Z,6,148,"[139, 140, 143, 147, 142, 148, 141]",23,6,148,...,81,2022-08-23 11:27:50.891,2022-08-23 11:28:45.618,0,0,True,True,56.632,54.727,503
1930,3,2022-08-23T11:29:32.908Z,No Feedback,2022-08-23T11:29:29.764Z,1,152,"[152, 139, 138, 134, 145, 144, 148]",23,1,152,...,82,2022-08-23 11:28:54.105,2022-08-23 11:29:28.940,0,0,True,True,37.968,34.835,504


'LayerStats:'

,BlockId,CommitResultDate,Condition,EndInteractionDate,ExpectedResultLayer,ExpectedResultNumber,LayerNumberConfiguration,ProbandId,ProvidedResultLayer,ProvidedResultNumber,...,CleanedUpInteractionStart_DT,CleanedUpInteractionEnd_DT,ResultNumberDifference,ResultLayerDifference,ResultLayerCorrect,ResultNumberCorrect,DurationMS,DurationMS_Cleaned,countCondition,LayerChange
0,-4,2022-08-01T14:01:37.953Z,Combined Feedback,2022-08-01T14:01:30.874Z,3,176,"[131, 144, 176, 153, 147, 135, 114]",2,2,153,...,2022-08-01 13:59:15.117,2022-08-01 14:01:30.874,-23,-1,False,False,135.757,135.757,0,60
1,-4,2022-08-01T14:02:26.995Z,Combined Feedback,2022-08-01T14:02:22.682Z,7,158,"[149, 118, 145, 137, 147, 146, 158]",2,4,147,...,2022-08-01 14:01:37.954,2022-08-01 14:02:22.682,-11,-3,False,False,44.728,44.728,1,60
2,-3,2022-08-01T14:03:44.338Z,Visual Feedback,2022-08-01T14:03:39.613Z,2,161,"[150, 161, 134, 151, 128, 149, 127]",2,2,161,...,2022-08-01 14:02:33.306,2022-08-01 14:03:39.613,0,0,True,True,66.307,66.307,0,60
3,-3,2022-08-01T14:04:34.107Z,Visual Feedback,2022-08-01T14:04:29.378Z,4,150,"[132, 144, 148, 150, 149, 147, 130]",2,4,150,...,2022-08-01 14:03:44.339,2022-08-01 14:04:29.378,0,0,True,True,45.039,45.039,1,60
4,-2,2022-08-01T14:07:33.355Z,Tactile Feedback,2022-08-01T14:07:23.779Z,6,160,"[139, 144, 128, 136, 156, 160, 137]",2,3,160,...,2022-08-01 14:05:08.405,2022-08-01 14:07:23.779,0,-3,False,True,135.374,135.374,0,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927,3,2022-08-23T11:27:13.692Z,No Feedback,2022-08-23T11:27:09.628Z,7,159,"[153, 122, 128, 144, 140, 154, 159]",23,7,159,...,2022-08-23 11:26:33.878,2022-08-23 11:27:08.564,0,0,True,True,37.272,34.686,501,21
1928,3,2022-08-23T11:27:49.516Z,No Feedback,2022-08-23T11:27:45.172Z,5,157,"[146, 142, 133, 156, 157, 128, 138]",23,5,157,...,2022-08-23 11:27:15.467,2022-08-23 11:27:44.655,0,0,True,True,31.479,29.188,502,18
1929,3,2022-08-23T11:28:51.795Z,No Feedback,2022-08-23T11:28:46.148Z,6,148,"[139, 140, 143, 147, 142, 148, 141]",23,6,148,...,2022-08-23 11:27:50.891,2022-08-23 11:28:45.618,0,0,True,True,56.632,54.727,503,26
1930,3,2022-08-23T11:29:32.908Z,No Feedback,2022-08-23T11:29:29.764Z,1,152,"[152, 139, 138, 134, 145, 144, 148]",23,1,152,...,2022-08-23 11:28:54.105,2022-08-23 11:29:28.940,0,0,True,True,37.968,34.835,504,27


'Numbers:'

...

,BlockId,Condition,ConditionId,TrialId,ResultErrorOR
0,-4,Combined Feedback,3,0,1
1,-4,Combined Feedback,3,1,1
2,-3,Visual Feedback,2,0,0
3,-3,Visual Feedback,2,1,0
4,-2,Tactile Feedback,1,0,1
...,...,...,...,...,...
1927,3,No Feedback,0,16,0
1928,3,No Feedback,0,17,0
1929,3,No Feedback,0,18,0
1930,3,No Feedback,0,19,0


'SPSS EXPORT (ERROR Sum):'

,Condition,ConditionId,TrialId,ResultErrorOR
0,Combined Feedback,3,0,1
1,Combined Feedback,3,1,2
2,Combined Feedback,3,2,1
3,Combined Feedback,3,3,0
4,Combined Feedback,3,4,1
...,...,...,...,...
79,Visual Feedback,2,16,1
80,Visual Feedback,2,17,1
81,Visual Feedback,2,18,3
82,Visual Feedback,2,19,2


'LayerStats per Layer:'

,BlockId,CommitResultDate,Condition,EndInteractionDate,ExpectedResultLayer,ExpectedResultNumber,LayerNumberConfiguration,ProbandId,ProvidedResultLayer,ProvidedResultNumber,...,CleanedUpInteractionEnd_DT,ResultNumberDifference,ResultLayerDifference,ResultLayerCorrect,ResultNumberCorrect,DurationMS,DurationMS_Cleaned,countCondition,Layer,LayerChange
0,-4,2022-08-01T14:01:37.953Z,Combined Feedback,2022-08-01T14:01:30.874Z,3,176,"[131, 144, 176, 153, 147, 135, 114]",2,2,153,...,2022-08-01 14:01:30.874,-23,-1,False,False,135.757,135.757,0,0,6
1,-4,2022-08-01T14:01:37.953Z,Combined Feedback,2022-08-01T14:01:30.874Z,3,176,"[131, 144, 176, 153, 147, 135, 114]",2,2,153,...,2022-08-01 14:01:30.874,-23,-1,False,False,135.757,135.757,0,1,17
2,-4,2022-08-01T14:01:37.953Z,Combined Feedback,2022-08-01T14:01:30.874Z,3,176,"[131, 144, 176, 153, 147, 135, 114]",2,2,153,...,2022-08-01 14:01:30.874,-23,-1,False,False,135.757,135.757,0,2,14
3,-4,2022-08-01T14:01:37.953Z,Combined Feedback,2022-08-01T14:01:30.874Z,3,176,"[131, 144, 176, 153, 147, 135, 114]",2,2,153,...,2022-08-01 14:01:30.874,-23,-1,False,False,135.757,135.757,0,3,9
4,-4,2022-08-01T14:01:37.953Z,Combined Feedback,2022-08-01T14:01:30.874Z,3,176,"[131, 144, 176, 153, 147, 135, 114]",2,2,153,...,2022-08-01 14:01:30.874,-23,-1,False,False,135.757,135.757,0,4,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15411,3,2022-08-23T11:30:05.548Z,No Feedback,2022-08-23T11:30:01.812Z,7,169,"[149, 122, 137, 130, 150, 143, 169]",23,7,169,...,2022-08-23 11:30:00.879,0,0,True,True,28.904,26.450,505,3,1
15412,3,2022-08-23T11:30:05.548Z,No Feedback,2022-08-23T11:30:01.812Z,7,169,"[149, 122, 137, 130, 150, 143, 169]",23,7,169,...,2022-08-23 11:30:00.879,0,0,True,True,28.904,26.450,505,4,2
15413,3,2022-08-23T11:30:05.548Z,No Feedback,2022-08-23T11:30:01.812Z,7,169,"[149, 122, 137, 130, 150, 143, 169]",23,7,169,...,2022-08-23 11:30:00.879,0,0,True,True,28.904,26.450,505,5,1
15414,3,2022-08-23T11:30:05.548Z,No Feedback,2022-08-23T11:30:01.812Z,7,169,"[149, 122, 137, 130, 150, 143, 169]",23,7,169,...,2022-08-23 11:30:00.879,0,0,True,True,28.904,26.450,505,6,3


## Aggregate Block-Level Result Rates
Group data by participant and block, compute counts/ratios and mean duration metrics, and export block-level summary tables for SPSS.

In [9]:
grp_block = results_spss.groupby(['ProbandId','BlockId']).count()

grp_sum = results_spss.groupby(['ProbandId','BlockId']).sum()

grp_mean = results_spss.groupby(['ProbandId','BlockId']).mean()

reset = grp_block.reset_index()

ratio = pd.DataFrame() 

ratio['ProbandId'] = reset['ProbandId']
ratio['BlockId'] = reset['BlockId']

ratio['NumTrials'] = reset['TrialNumber']

display(grp_sum['ResultLayerCorrect'].reset_index())

merged_sum = pd.merge(ratio, results_spss[['ProbandId','BlockId', 'Condition']]).drop_duplicates()
merged_sum.reset_index(inplace=True, drop=True)

merged_sum['ConditionId'] = merged_sum['Condition'].map(condition_names.index)

merged_sum = pd.merge(merged_sum, grp_sum[['ResultLayerCorrect', 'ResultNumberCorrect', 'ResultLayerDifference', 'ResultLayerDifference_Abs', 'ResultLayerDifference_Proz', 'BothCorrect', 'OneCorrect', 'NoneCorrect']].reset_index()).drop_duplicates()
merged_sum.reset_index(inplace=True, drop=True)

merged_sum['ResultLayerCorrectCount'] = merged_sum['ResultLayerCorrect']
merged_sum.drop('ResultLayerCorrect', axis=1, inplace=True)
merged_sum['ResultLayerCorrectRatio'] = merged_sum['ResultLayerCorrectCount'] / merged_sum['NumTrials']

merged_sum['ResultNumberCorrectCount'] = merged_sum['ResultNumberCorrect']
merged_sum.drop('ResultNumberCorrect', axis=1, inplace=True)
merged_sum['ResultNumberCorrectRatio'] = merged_sum['ResultNumberCorrectCount'] / merged_sum['NumTrials']

merged_sum['ResultLayerDifferenceRatio'] = merged_sum['ResultLayerDifference'] / merged_sum['NumTrials']
merged_sum['ResultLayerDifferenceRatio_Abs'] = merged_sum['ResultLayerDifference_Abs'] / merged_sum['NumTrials']
merged_sum['ResultLayerDifferenceRatio_Proz'] = merged_sum['ResultLayerDifference_Proz'] / merged_sum['NumTrials']

merged_sum['ResultLayerErrorCount'] = merged_sum['NumTrials'] - merged_sum['ResultLayerCorrectCount']
merged_sum['ResultNumberErrorCount'] = merged_sum['NumTrials'] - merged_sum['ResultNumberCorrectCount']

merged_sum['ResultErrorLayerANDNumber'] = merged_sum['NoneCorrect']
merged_sum['ResultErrorLayerXORNumber'] = merged_sum['OneCorrect']
merged_sum['ResultErrorLayerORNumber'] = merged_sum['OneCorrect'] + merged_sum['NoneCorrect']

merged_sum.drop('NumTrials', axis=1, inplace=True)

merged_sum = pd.merge(merged_sum, grp_mean['TrialDurationS'].reset_index()).drop_duplicates()
merged_sum['TrialDurationMeanS'] = merged_sum['TrialDurationS']
merged_sum.drop('TrialDurationS', axis=1, inplace=True)

merged_sum_experiment = merged_sum[merged_sum['BlockId'] >= 0]

merged_sum.to_csv(rf'{export_data_spss}result_rates_complete.csv', sep= ";")
merged_sum_experiment.to_csv(rf'{export_data_spss}result_rates_experiment.csv', sep= ";")


display(merged_sum)

,ProbandId,BlockId,ResultLayerCorrect
0,2,-4,0
1,2,-3,2
2,2,-2,1
3,2,-1,2
4,2,0,20
...,...,...,...
163,23,-1,1
164,23,0,17
165,23,1,15
166,23,2,15


,ProbandId,BlockId,Condition,ConditionId,ResultLayerDifference,ResultLayerDifference_Abs,ResultLayerDifference_Proz,BothCorrect,OneCorrect,NoneCorrect,...,ResultNumberCorrectRatio,ResultLayerDifferenceRatio,ResultLayerDifferenceRatio_Abs,ResultLayerDifferenceRatio_Proz,ResultLayerErrorCount,ResultNumberErrorCount,ResultErrorLayerANDNumber,ResultErrorLayerXORNumber,ResultErrorLayerORNumber,TrialDurationMeanS
0,2,-4,Combined Feedback,3,-4,4,0.571429,0,0,2,...,0.000000,-2.000000,2.000000,0.285714,2,2,2,0,2,90.242500
1,2,-3,Visual Feedback,2,0,0,0.000000,2,0,0,...,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,55.673000
2,2,-2,Tactile Feedback,1,-3,3,0.428571,0,2,0,...,0.500000,-1.500000,1.500000,0.214286,1,1,0,2,2,98.446000
3,2,-1,No Feedback,0,0,0,0.000000,2,0,0,...,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,76.752000
4,2,0,No Feedback,0,-1,1,0.142857,20,1,0,...,1.000000,-0.047619,0.047619,0.006803,1,0,0,1,1,41.977857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,23,-1,No Feedback,0,-5,5,0.714286,1,0,1,...,0.500000,-2.500000,2.500000,0.357143,1,1,1,0,1,37.796000
164,23,0,Visual Feedback,2,-9,9,1.285714,16,2,3,...,0.809524,-0.428571,0.428571,0.061224,4,4,3,2,5,41.050667
165,23,1,Combined Feedback,3,4,10,1.428571,14,6,1,...,0.904762,0.190476,0.476190,0.068027,6,2,1,6,7,33.784857
166,23,2,Tactile Feedback,1,8,22,3.142857,15,2,4,...,0.809524,0.380952,1.047619,0.149660,6,4,4,2,6,29.682476


## Create Trial-Wise Repeated-Measures Matrix
Pivot experiment trials into a wide format by participant, condition, and trial; add binary visual/tactile indicators; and export the repeated-measures table.

In [27]:
pivot = results_spss_experiment.pivot(index=['ProbandId', 'Condition'], columns=['TrialId'], values=['ResultLayerCorrect', 'ResultLayerDifference', 'ResultLayerDifference_Abs', 'ResultLayerDifference_Proz', 'ResultNumberCorrect', 'ResultNumberDifference', 'TrialDurationS', 'LayerChangeCount', 'ShowNumberCount', 'ShowNumberDurationMS', 'CenterDurationMS']).reset_index()
pivot['ConditionId'] = pivot['Condition'].map(condition_names.index)
col = pivot.pop('ConditionId')
pivot.insert(1, col.name, col)

pivot['Visual'] = 0
pivot['Tactile'] = 0
# visual feedback = 2, combined = 3 
pivot.loc[pivot['ConditionId'] > 1, 'Visual'] = 1
# tactile feedback = 1
pivot.loc[pivot['ConditionId'] == 1, 'Tactile'] = 1
# combined = 3
pivot.loc[pivot['ConditionId'] == 3, 'Tactile'] = 1

col = pivot.pop('Visual')
pivot.insert(3, col.name, col)

col = pivot.pop('Tactile')
pivot.insert(4, col.name, col)

display(pivot)

pivot.to_csv(rf'{export_data_spss}results_experiment_repeat.csv', sep= ";")

ProbandId ConditionId          Condition Visual Tactile  \
TrialId                                                           
0               2           3  Combined Feedback      1       1   
1               2           0        No Feedback      0       0   
2               2           1   Tactile Feedback      0       1   
3               2           2    Visual Feedback      1       0   
4               4           3  Combined Feedback      1       1   
..            ...         ...                ...    ...     ...   
79             22           2    Visual Feedback      1       0   
80             23           3  Combined Feedback      1       1   
81             23           0        No Feedback      0       0   
82             23           1   Tactile Feedback      0       1   
83             23           2    Visual Feedback      1       0   

        ResultLayerCorrect                      ... CenterDurationMS  \
TrialId                  0    1    2    3    4  ...               11   
0                      1.0  0.0  1.0  1.0  1.0  ...      1301.285714   
1                      1.0  1.0  0.0  1.0  1.0  ...      1029.222222   
2                      1.0  1.0  1.0  1.0  1.0  ...      1518.333333   
3                      1.0  1.0  1.0  0.0  1.0  ...      1153.166667   
4                      1.0  1.0  1.0  1.0  1.0  ...      1144.900000   
..                     ...  ...  ...  ...  ...  ...              ...   
79                     1.0  1.0  1.0  1.0  1.0  ...      1365.571429   
80                     1.0  1.0  1.0  1.0  1.0  ...      1389.444444   
81                     1.0  1.0  1.0  0.0  0.0  ...      1105.000000   
82                     1.0  0.0  1.0  1.0  0.0  ...      1311.375000   
83                     1.0  1.0  1.0  1.0  0.0  ...      1494.625000   

                                                                          \
TrialId           12           13           14           15           16   
0        1366.857143  1057.142857  1086.750000  1290.000000   991.000000   
1        1293.500000  1199.142857  1810.166667  1154.571429  1618.250000   
2        1492.857143  1298.142857  1185.142857  1047.500000  1546.125000   
3        1052.571429  1128.000000  1295.285714  1412.714286  1065.000000   
4        1333.181818  1328.888889  1447.555556  1103.111111  1163.000000   
..               ...          ...          ...          ...          ...   
79       1467.857143  1759.714286  1501.571429  1665.142857  1721.142857   
80       1518.866667  1448.437500  1595.222222  1334.666667  1892.750000   
81       1415.111111  1590.250000  1749.300000  2021.000000  1496.714286   
82       1114.416667  1262.222222  1333.333333  1665.750000  1248.285714   
83       1321.555556  1390.266667  1811.900000  1488.750000  1510.250000   

                                                             
TrialId           17           18           19           20  
0        1620.100000  1141.714286  1222.285714  1545.250000  
1        1614.818182  1459.727273  1054.181818  1163.000000  
2        1472.142857  1393.600000  1425.875000  1200.444444  
3        1893.285714  1487.416667  1834.500000  1241.857143  
4        1220.375000  1192.444444  1242.625000  2097.500000  
..               ...          ...          ...          ...  
79       1582.000000  1715.571429  1733.857143  1753.714286  
80       1171.200000  1202.500000  1301.000000  1736.111111  
81       1334.333333  1570.692308  1273.000000  1485.857143  
82       1508.500000  1631.857143  1154.750000  1618.571429  
83       1984.285714  1842.500000  1171.125000  1593.250000  

[84 rows x 236 columns]

## Create Participant-Wise Wide Pivot (Inspection)
Generate an alternative participant-level wide pivot across condition/trial combinations for quick inspection and downstream analysis experiments.

In [33]:
results_spss_experiment['ConditionId'] = results_experiment['Condition'].map(condition_names.index)

results_experiment['Visual'] = 0
results_experiment['Tactile'] = 0
# visual feedback = 2, combined = 3 
results_experiment.loc[results_experiment['ConditionId'] > 1, 'Visual'] = 1
# tactile feedback = 1
results_experiment.loc[results_experiment['ConditionId'] == 1, 'Tactile'] = 1
# combined = 3
results_experiment.loc[results_experiment['ConditionId'] == 3, 'Tactile'] = 1

pivot = results_spss_experiment.pivot(index=['ProbandId'], columns=['Condition', 'TrialId',], values=['ResultLayerCorrect', 'ResultLayerDifference', 'ResultLayerDifference_Abs', 'ResultLayerDifference_Proz', 'ResultNumberCorrect', 'ResultNumberDifference', 'TrialDurationS', 'LayerChangeCount', 'ShowNumberCount', 'ShowNumberDurationMS', 'CenterDurationMS']).reset_index()
# pivot = pivot.reset_index(drop=True).reset_index(drop=True).reset_index(drop=True)
display(pivot)

<ipython-input-33-f3dd78455dfd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_spss_experiment['ConditionId'] = results_experiment['Condition'].map(condition_names.index)
<ipython-input-33-f3dd78455dfd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_experiment['Visual'] = 0
<ipython-input-33-f3dd78455dfd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

ProbandId ResultLayerCorrect                                     \
Condition                  No Feedback                                      
TrialId                              0    1    2    3    4    5    6    7   
0                 2                1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0   
1                 4                1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
2                 5                1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
3                 6                1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
4                 7                1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
5                 8                1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
6                 9                1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
7                10                1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
8                11                1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
9                12                1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
10               13                1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0   
11               14                1.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0   
12               15                1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
13               16                1.0  1.0  0.0  1.0  1.0  1.0  0.0  1.0   
14               17                0.0  0.0  1.0  1.0  1.0  0.0  1.0  1.0   
15               18                1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
16               19                1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0   
17               20                1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
18               21                1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
19               22                1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0   
20               23                1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0   

                ... CenterDurationMS                                         \
Condition       ... Tactile Feedback                                          
TrialId      8  ...               11           12           13           14   
0          1.0  ...      1518.333333  1492.857143  1298.142857  1185.142857   
1          1.0  ...      2659.200000  1997.900000  1298.857143  1109.125000   
2          1.0  ...      2067.333333  1426.111111  1367.500000  1459.285714   
3          1.0  ...      1583.700000  1746.714286  1927.428571  1758.500000   
4          1.0  ...      2101.375000  2709.285714  1672.700000  2757.500000   
5          1.0  ...      1143.777778  1509.375000  1221.222222  1479.714286   
6          1.0  ...      2027.375000  1476.000000  1493.571429  1474.750000   
7          1.0  ...      1412.300000  1324.285714  1132.571429  1358.428571   
8          1.0  ...      1422.000000  1631.727273  1423.888889  1133.700000   
9          1.0  ...      1615.125000  1531.714286  1693.333333  1493.555556   
10         1.0  ...      1144.000000  1226.444444  1929.571429  1827.000000   
11         1.0  ...      1393.538462  1955.000000  1623.133333  1487.000000   
12         1.0  ...      1388.125000  1379.888889  1199.142857  1483.454545   
13         0.0  ...      1310.500000  2552.125000  1048.000000  1178.875000   
14         1.0  ...      1868.133333  2160.250000  2157.111111  1744.307692   
15         1.0  ...      1038.000000  1303.076923  1708.375000  1327.000000   
16         1.0  ...      1575.625000  1631.444444  1291.090909  2090.100000   
17         1.0  ...      2161.000000  1624.555556  2070.800000  1820.000000   
18         1.0  ...      1704.666667  1948.454545  1585.818182  1693.642857   
19         1.0  ...      1809.375000  1660.125000  1398.142857  1284.250000   
20         1.0  ...      1311.375000  1114.416667  1262.222222  1333.333333   

                                                                            \
Condition                                                                    
TrialId             15           16           17           18           19   
0          1047.500000  1546.1